In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split  # Import train_test_split
from sklearn import metrics
# Load the dataset
d=pd.read_csv('data/cardio_train.csv',delimiter=';')

feature_cols1 = ['age', 'gender',"height", 'weight','ap_lo','ap_hi','cholesterol',"gluc","smoke","active"]
logistic_regression = LogisticRegression()

In [31]:
d["weight"]=d["weight"]/74.205690
d["age"]=d["age"]/29
d["height"]=d["height"]/164.359229

In [32]:
Q1 = d["ap_lo"].quantile(0.25)
Q3 = d["ap_lo"].quantile(0.75)
IQR = Q3 - Q1

d["ap_lo"] = d["ap_lo"][~((d["ap_lo"] < (Q1 - 1.5 * IQR)) |(d["ap_lo"] > (Q3 + 1.5 * IQR)))]

In [33]:
Q1 = d["ap_lo"].quantile(0.25)
Q3 = d["ap_lo"].quantile(0.75)
IQR = Q3 - Q1

d["ap_lo"] = d["ap_lo"][~((d["ap_lo"] < (Q1 - 1.5 * IQR)) |(d["ap_lo"] > (Q3 + 1.5 * IQR)))]

In [34]:
d["ap_lo"]=d["ap_lo"].fillna(d["ap_lo"].mean())
d["ap_hi"]=d["ap_hi"].fillna(d["ap_hi"].mean())

In [35]:
X = d[feature_cols1] # Features
y = d.cardio 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=4)

In [36]:
import mlflow
from sklearn.metrics import mean_squared_error, r2_score
with mlflow.start_run():
    logistic_regression= LogisticRegression()
    logistic_regression.fit(X_train,y_train)
    y_pred=logistic_regression.predict(X_test)
    mlflow.log_metric('Accuracy',metrics.accuracy_score(y_test, y_pred))
    mlflow.log_metric('MSE',mean_squared_error(y_test, y_pred))
    mlflow.log_metric('r2',r2_score(y_test, y_pred))
    mlflow.sklearn.log_model(logistic_regression, "model", registered_model_name="logistic-regression_demo")

/home/lentraadmin/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
Registered model 'logistic-regression_demo' already exists. Creating a new version of this model...
2023/04/12 12:26:04 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: logistic-regression_demo, version 4
Created version '4' of model 'logistic-regression_demo'.


In [37]:
mlflow.end_run()

In [38]:
from mlflow import MlflowClient

client = MlflowClient()
client.create_registered_model("logistic_regression-model-1")

MlflowException: Registered Model (name=logistic_regression-model-1) already exists.